# Tarefa 2a: resumo de textos com arquivos pequenos com o Titan Text Premier


Neste caderno, você vai ingerir uma pequena sequência de texto diretamente na API do Amazon Bedrock (usando o modelo do Titan Text) e vai instruí-la a resumir o texto de entrada. Você pode aplicar essa abordagem para resumir transcrições de chamadas, transcrições de reuniões, livros, artigos, publicações de blogs e outros conteúdos relevantes quando o tamanho do texto de entrada estiver dentro dos limites de tamanho do contexto do modelo.

## Tarefa 2a.1: configurar o ambiente

Nesta tarefa, você configurará o ambiente.

In [34]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore
from IPython.display import Markdown, display

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## Tarefa 2a.2: escrever prompt com o texto a ser resumido

Nesta tarefa, você usará um pequeno trecho do texto com menos tokens do que o tamanho máximo permitido pelo modelo de base. Como exemplo de texto de entrada para este laboratório, você usa um parágrafo de uma [postagem do Blog da AWS] (https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) que anuncia o Amazon Bedrock.

O prompt começa com uma instrução `Please provide a summary of the following text.`. 

In [35]:
prompt_data = """

Por favor, forneça um resumo do seguinte texto:

A AWS analisou todos os feedbacks dos clientes e hoje temos o prazer de anunciar o Amazon Bedrock, \
um novo serviço que torna os FMs da AI21 Labs, Anthropic, Stability AI e Amazon acessíveis por meio de uma API. \
O Bedrock é a maneira mais fácil para os clientes criarem e escalarem aplicativos baseados em IA generativa usando FMs, \
democratizando o acesso para todos os desenvolvedores. \
O Bedrock oferecerá a capacidade de acessar uma variedade de FMs poderosos para texto e imagens \
 — incluindo os FMs Titan da Amazon, que consistem em dois novos LLMs que também estamos anunciando hoje \
 — por meio de um serviço gerenciado da AWS escalável, confiável e seguro. Com a experiência serverless da Bedrock, \
os clientes podem encontrar facilmente o modelo certo para o que estão tentando realizar, começar rapidamente, \
personalizar os FMs de forma privada com seus próprios dados e integrá-los e implantá-los facilmente em seus \
aplicativos usando as ferramentas e recursos da AWS com os quais já estão familiarizados, sem precisar gerenciar \
nenhuma infraestrutura (incluindo integrações com recursos do Amazon SageMaker ML, como Experimentos para testar \
diferentes modelos e Pipelines para gerenciar seus FMs em escala).
"""

## Tarefa 2a.3: Criar corpo da solicitação com parâmetros de inferência e prompt 

Nesta tarefa, você criará o corpo do prompt com os parâmetros de inferência e prompt acima.

In [51]:
# request body - Amazon Nova format
body = json.dumps({
    "messages": [
        {
            "role": "user",
            "content": [{"text": prompt_data}]
        }
    ],
    "inferenceConfig": {
        "maxTokens": 1000,
        "temperature": 0.7
    }
})

## Tarefa 2a.4: Invocar o modelo de base via Boto3

Nesta tarefa, você enviará uma solicitação de API para o Amazon Bedrock especificando os parâmetros da solicitação: `modelId`, `accept` e `contentType`. Após o prompt fornecido, o modelo de base no Amazon Bedrock resume o texto de entrada.

### Concluir geração de saída

Por padrão, o serviço Amazon Bedrock gera o resumo completo de um determinado prompt em uma única saída. Pode demorar se a saída do modelo contiver muitos tokens. 

In [53]:
#model configuration and invoke the model
modelId = 'amazon.nova-lite-v1:0' # Amazon Nova 
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:
    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    
    # Nova usa formato diferente - verifique a estrutura da resposta
    print("Estrutura da resposta:", response_body.keys())
    
    # Tente extrair o texto (formato pode variar)
    if 'output' in response_body:
        outputText = response_body['output']['message']['content'][0]['text']
    elif 'outputText' in response_body:
        outputText = response_body['outputText']
    else:
        print("Resposta completa:", response_body)
        
    #print(outputText)
    display(Markdown(outputText))

except Exception as e:
    print(f"Erro: {e}")

Estrutura da resposta: dict_keys(['output', 'stopReason', 'usage'])


A AWS anunciou o lançamento do Amazon Bedrock, um novo serviço que facilita o acesso a modelos de inteligência artificial (IA) de empresas como AI21 Labs, Anthropic, Stability AI e da própria Amazon por meio de uma API. O Bedrock visa democratizar o uso de IA generativa, permitindo que desenvolvedores criem e escalem aplicativos com modelos de linguagem (LLMs) e modelos de imagem de forma rápida e segura. O serviço oferece modelos gerenciados pela AWS, incluindo os novos modelos Titan da Amazon, e elimina a necessidade de gerenciar infraestrutura. Com integrações com o Amazon SageMaker, os usuários podem testar e gerenciar modelos em escala.

In [42]:
# Exibir contagem de tokens
if 'usage' in response_body:
    input_tokens = response_body['usage'].get('inputTokens', 0)
    output_tokens = response_body['usage'].get('outputTokens', 0)
    total_tokens = input_tokens + output_tokens
    
    print(f"\n{'='*50}")
    print(f"📊 RESUMO DE TOKENS")
    print(f"{'='*50}")
    print(f"Tokens de entrada:  {input_tokens}")
    print(f"Tokens de saída:    {output_tokens}")
    print(f"Total:              {total_tokens}")
    print(f"{'='*50}\n")


📊 RESUMO DE TOKENS
Tokens de entrada:  289
Tokens de saída:    184
Total:              473



### Transmitir geração de saída

Em seguida, você verá como usar a API invoke_model_with_response_stream do Amazon Bedrock para transmitir saídas do modelo para que os usuários possam consumi-las conforme são geradas. Em vez de gerar a saída completa de uma só vez, essa API retorna um ResponseStream que envia blocos de saída menores do modelo conforme são produzidos. Você pode exibir essas saídas de transmissão em uma visualização contínua e consumível.

In [44]:
#invoke model with response stream
modelId = 'amazon.nova-lite-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)

stream = response.get('body')
output = list(stream)
output

[{'chunk': {'bytes': b'{"messageStart":{"role":"assistant"}}'}},
 {'chunk': {'bytes': b'{"contentBlockDelta":{"delta":{"text":"A"},"contentBlockIndex":0}}'}},
 {'chunk': {'bytes': b'{"contentBlockStop":{"contentBlockIndex":0}}'}},
 {'chunk': {'bytes': b'{"contentBlockDelta":{"delta":{"text":" AWS anuncio"},"contentBlockIndex":1}}'}},
 {'chunk': {'bytes': b'{"contentBlockStop":{"contentBlockIndex":1}}'}},
 {'chunk': {'bytes': b'{"contentBlockDelta":{"delta":{"text":"u o"},"contentBlockIndex":2}}'}},
 {'chunk': {'bytes': b'{"contentBlockStop":{"contentBlockIndex":2}}'}},
 {'chunk': {'bytes': b'{"contentBlockDelta":{"delta":{"text":" Amazon Bedrock, um"},"contentBlockIndex":3}}'}},
 {'chunk': {'bytes': b'{"contentBlockStop":{"contentBlockIndex":3}}'}},
 {'chunk': {'bytes': b'{"contentBlockDelta":{"delta":{"text":" servi\xc3\xa7o"},"contentBlockIndex":4}}'}},
 {'chunk': {'bytes': b'{"contentBlockStop":{"contentBlockIndex":4}}'}},
 {'chunk': {'bytes': b'{"contentBlockDelta":{"delta":{"text"

In [48]:
from IPython.display import display_markdown,Markdown,clear_output

In [71]:
modelId = 'amazon.nova-lite-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1

print("Processando stream da resposta...\n")
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            
            # Processa contentBlockDelta (texto real)
            if 'contentBlockDelta' in chunk_obj:
                text = chunk_obj['contentBlockDelta']['delta']['text']
                output.append(text)
                i += 1
            
            # Processa metadata (tokens)
            elif 'metadata' in chunk_obj:
                print(f"Total tokens: {usage.get('inputTokens', 0) + usage.get('outputTokens', 0)}")

# Exibir saída completa formatada
print('\n\t\t\x1b[31m**RESUMO COMPLETO**\x1b[0m\n')
complete_output = ''.join(output)
display(Markdown(complete_output))

Processando stream da resposta...

Total tokens: 458

		**RESUMO COMPLETO**



A AWS anunciou o lançamento do Amazon Bedrock, um serviço que facilita o acesso e uso de modelos de inteligência artificial (IA) de diversas empresas, incluindo AI21 Labs, Anthropic, Stability AI e Amazon, através de uma API. Este serviço tem como objetivo democratizar o uso de IA generativa para desenvolvedores de todas as habilidades, permitindo que eles criem e escalem aplicativos baseados em IA de forma mais fácil. O Bedrock oferece acesso a uma variedade de modelos de linguagem (LLMs) e de geração de imagens, incluindo os novos modelos Titan da Amazon, tudo gerenciado por um serviço escalável, confiável e seguro da AWS. Os usuários podem facilmente encontrar modelos adequados para suas necessidades, começar rapidamente, personalizar os modelos com seus próprios dados e integrar esses modelos em seus aplicativos usando as ferramentas e recursos familiares da AWS, sem a necessidade de gerenciar a infraestrutura subjacente. O serviço também inclui integrações com recursos do Amazon SageMaker para testes e gerenciamento de modelos em escala.

Você testou o uso do SDK boto3 para acessar a API do Amazon Bedrock. Esse SDK fornece acesso programático básico aos recursos do Bedrock. Ao usar essa API, você conseguiu implementar dois casos de uso: 1) Gerar um resumo de texto completo do conteúdo de notícias da AWS de uma só vez e 2) Transmitir a saída resumida em blocos para processamento incremental.

### Experimente você mesmo
- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com **Task2b.ipynb**.